<br>

# 데이터 전처리
- 분포 양 끝단의 0.2%만큼의 데이터를 2분위, 3분위의 값으로 변경(이상치의 특성 약화시키기 위한 목적)

In [2]:
# 패키지 임포트
import pandas as pd

# 데이터 불러오기
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
sample_submission = pd.read_csv('sample_submission.csv', index_col=0)

# Train 데이터의 타입을 Sample_submission에 대응하는 가변수 형태로 변환
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x : to_number(x, column_number))

# 모델에 적용할 데이터 셋 준비 
train_x = train.drop(columns=['type', 'type_num'], axis=1)
train_y = train['type_num']
test_x = test

In [3]:
# 이중 for문으로 만들어야함(열 for문으로 넣고 그 안에 행에대한 for문 삽입)

def outlier_to_iqr(x):
    for col in range(0, len(x.columns), +1):                            # 열에 대한 반복문
        Q1, Q3 = np.percentile(x.iloc[:, col:col+1], [0.2, 99.8])       # 분포 양 끝의 값을 변경
        
        for idx in range(0, len(x), +1):                                # 행에 대한 반복문
            if float(x.iloc[idx, col:col+1]) < Q1:                      # 만약 2분위 수 보다 작다면
                x.iloc[idx, col:col+1] = Q1                             # 2분위의 값으로
            elif float(x.iloc[idx, col:col+1]) > Q3:                    # 3분위 보다 크다면
                x.iloc[idx, col:col+1] = Q3                             # 3분위의 값으로
    return x

In [4]:
# train 데이터 변경
import numpy as np

train_x = outlier_to_iqr(train_x)

In [5]:
train_x.describe()

,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
count,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,...,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000
mean,360.822862,20.984007,19.860607,19.286211,18.870394,18.612132,21.171277,20.089070,19.503911,19.085584,...,20.714714,19.600703,19.007003,18.601740,18.393617,20.731236,19.531210,18.939749,18.519284,18.276422
std,225.278844,2.068628,1.853275,1.696300,1.613456,1.674223,1.956686,1.834191,1.698218,1.620603,...,2.323579,2.028606,1.897642,1.839384,1.985542,2.176587,1.951686,1.852092,1.785942,1.855230
min,2.000000,16.268373,15.219741,15.067677,14.915361,14.542514,16.591009,15.546216,15.413848,15.259058,...,15.560760,14.865154,14.335362,14.036789,13.709452,16.164638,14.962227,14.450383,14.114361,13.800042
25%,174.000000,19.652590,18.701180,18.048572,17.747663,17.425523,19.940403,18.902851,18.259352,17.903615,...,19.247795,18.113933,17.479794,17.050294,16.804705,19.266214,18.076120,17.423425,16.977671,16.705774
50%,349.000000,20.871359,19.904235,19.454492,19.043895,18.611799,21.049096,20.069038,19.631419,19.188763,...,20.366848,19.586559,19.182789,18.693370,18.174592,20.406840,19.547674,19.143156,18.641756,18.100997
75%,526.000000,22.160432,21.150297,20.515936,20.073528,19.883760,22.337540,21.385830,20.773911,20.331419,...,21.797480,21.004397,20.457491,20.019112,19.807652,21.992898,20.962386,20.408140,19.968846,19.819554
max,992.000000,26.938635,25.575422,24.059680,22.517061,23.315141,26.882039,25.706642,24.294442,22.870197,...,31.627548,27.897671,26.002954,25.279656,26.706507,27.289858,25.622653,23.959345,22.408437,23.273177


In [6]:
# 시간 절약을 위한 새로운 파일 생성
train_x.to_csv('train_x_0.2_99.8.csv', index=True)

In [ ]:
# Prescaling
import numpy as np
from sklearn import preprocessing

scaler = preprocessing.RobustScaler()
scaler.fit(train_x)

train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)

In [ ]:
print(train_x.shape, train_y.shape, test_x.shape)